# Exploring the effect of player position changes on predictions

Each season a number of players change position in the game. Notably for the 2020-21 season Aubamayang was changed from a forward to a midfielder. This has led to many people including him in their teams due to the extra point he scores for goals. However he was relatively low in our model's predictions.

For gameweek 1 our model uses the last 5 gameweeks of the previous season. Therefore old positions are used for predictions. In this notebook we explore the effect of changing historical total points and position for players who had a new position in 2020-21.

## Set-up

In [1]:
import os

import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [2]:
os.chdir('../..')

In [3]:
from src.data.s3_utilities import s3_filesystem

In [4]:
pd.options.display.max_columns = None

Taken from logs:

In [5]:
PLAYERS_WHO_CHANGED_POSITION_IN_2020_21 = {'ethan_ampadu', 'gabriel_teodoro_martinelli_silva', 'fernando_luiz_rosa', 'mason_greenwood', 'anthony_gordon', 'daniel_amartey', 'anthony_martial', 'kelland_watts', 'keinan_davis', 'marcus_rashford', 'pierre-emerick_aubameyang', 'john_lundstram', 'matt_ritchie', 'william_smallbone', 'michail_antonio', 'callum_robinson', 'diogo_jota', 'eric_dier', 'takumi_minamino', 'richarlison_de_andrade', 'ben_johnson', 'ryan_sessegnon'}

In [6]:
PREDICTIONS_GW1_S3_PATH = 's3://fpl-analysis-data/gw_predictions/season=2020-21/gw=1/2020-21-1.parquet'

## Predictions before position change

In [10]:
predictions_before_position_change = pq.read_table(PREDICTIONS_GW1_S3_PATH, filesystem=s3_filesystem).to_pandas()
predictions_before_position_change.head()

,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model
0,5.657178,5.443412,5.453287,5.351354,5.367841,27.273075,mohamed_salah,0.0,0.0,0.0,1.0,Liverpool,12.0,mohamed_salah,5.073999,5.093044,5.211539,5.165638,5.033789,25.578009,mohamed_salah,0.0,0.0,0.0,1.0,mohamed_salah,mohamed_salah,both,5.073999,5.093044,5.211539,5.165638,5.033789,25.578009,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
1,6.319860,6.077927,6.090129,5.983685,6.004195,30.475794,raheem_sterling,0.0,0.0,0.0,1.0,Manchester City,11.5,raheem_sterling,4.751492,5.080462,4.870038,4.858727,4.749552,24.310270,raheem_sterling,0.0,0.0,0.0,1.0,raheem_sterling,raheem_sterling,both,4.751492,5.080462,4.870038,4.858727,4.749552,24.310270,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
2,4.679602,4.493706,4.491284,4.391965,4.398650,22.455206,harry_kane,0.0,1.0,0.0,0.0,Tottenham Hotspur,10.5,harry_kane,4.643481,4.666337,4.685479,4.617877,4.522799,23.135973,harry_kane,0.0,1.0,0.0,0.0,harry_kane,harry_kane,both,4.643481,4.666337,4.685479,4.617877,4.522799,23.135973,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
3,4.239182,4.072062,4.067756,3.971863,3.975711,20.326572,andrew_robertson,1.0,0.0,0.0,0.0,Liverpool,7.0,andrew_robertson,5.085325,4.403040,4.415774,4.244756,4.093313,22.242208,andrew_robertson,1.0,0.0,0.0,0.0,andrew_robertson,andrew_robertson,both,5.085325,4.403040,4.415774,4.244756,4.093313,22.242208,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01
4,3.004309,2.883254,2.870653,2.782548,2.777399,14.318163,hugo_lloris,0.0,0.0,1.0,0.0,Tottenham Hotspur,5.5,hugo_lloris,4.492739,4.340151,4.312158,4.192936,4.086884,21.424868,hugo_lloris,0.0,0.0,1.0,0.0,hugo_lloris,hugo_lloris,both,4.492739,4.340151,4.312158,4.192936,4.086884,21.424868,0.0,0.0,1.0,0.0,DeepFantasyFootball_v01


In [13]:
predictions_before_position_change[predictions_before_position_change['name'].isin(PLAYERS_WHO_CHANGED_POSITION_IN_2020_21)].shape

(22, 39)

In [14]:
len(PLAYERS_WHO_CHANGED_POSITION_IN_2020_21)

22

`name_new` is the name as recorded in Fantasy Football Scout

In [19]:
old_predictions_players_who_changed_position = predictions_before_position_change.copy()[
    predictions_before_position_change['name'].isin(PLAYERS_WHO_CHANGED_POSITION_IN_2020_21)
][
    ['name', 'name_new', 'model', 'GW_plus_1', 'GW_plus_2', 'GW_plus_3', 'GW_plus_4', 'GW_plus_5', 'sum', 'position_DEF', 'position_FWD', 'position_GK', 'position_MID']
]

old_predictions_players_who_changed_position.head()

,name,name_new,model,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID
12,anthony_martial,anthony_martial,DeepFantasyFootball_v01,4.499953,4.105354,4.164009,4.055518,3.967909,20.792742,0.0,1.0,0.0,0.0
15,pierre-emerick_aubameyang,pierre-emerick_aubameyang,DeepFantasyFootball_v01,3.897748,4.059285,3.870273,3.847038,3.696694,19.371038,0.0,0.0,0.0,1.0
16,michail_antonio,michail_antonio,DeepFantasyFootball_v01,3.471481,3.811976,4.013702,4.029356,4.043107,19.369621,0.0,1.0,0.0,0.0
22,marcus_rashford,marcus_rashford,DeepFantasyFootball_v01,3.857534,3.467133,3.514597,3.449275,3.403746,17.692284,0.0,0.0,0.0,1.0
59,mason_greenwood,mason_greenwood,DeepFantasyFootball_v01,3.280328,2.741984,2.706571,2.626166,2.598459,13.953507,0.0,0.0,0.0,1.0


Only make the change for the new model predictions

In [21]:
old_predictions_players_who_changed_position = old_predictions_players_who_changed_position[old_predictions_players_who_changed_position['model'] == 'DeepFantasyFootball_v01']

## Make a dictionary of players and their new position

In [25]:
old_predictions_players_who_changed_position['position'] = np.nan

for position in ['DEF', 'FWD', 'GK', 'MID']:
    old_predictions_players_who_changed_position['position'] = np.where(
        old_predictions_players_who_changed_position[f'position_{position}'] == 1,
        position,
        old_predictions_players_who_changed_position['position']
    )

In [26]:
old_predictions_players_who_changed_position['position'].isnull().sum()

0

In [27]:
old_predictions_players_who_changed_position['position'].value_counts()

MID    9
DEF    5
FWD    5
Name: position, dtype: int64

In [29]:
dict(zip(old_predictions_players_who_changed_position['name_new'], old_predictions_players_who_changed_position['position']))

{'anthony_martial': 'FWD',
 'pierre-emerick_aubameyang': 'MID',
 'michail_antonio': 'FWD',
 'marcus_rashford': 'MID',
 'mason_greenwood': 'MID',
 'richarlison_de andrade': 'FWD',
 'matt_ritchie': 'MID',
 'john_lundstram': 'MID',
 'ben_johnson': 'DEF',
 'william_smallbone': 'MID',
 'fernando_luiz rosa': 'DEF',
 'takumi_minamino': 'MID',
 'eric_dier': 'DEF',
 'keinan_davis': 'FWD',
 'anthony_gordon': 'MID',
 'kelland_watts': 'DEF',
 'gabriel_teodoro martinelli silva': 'MID',
 'kouassi_ryan sessegnon': 'DEF',
 'callum_robinson': 'FWD'}

## Predictions after position change

We made code changes in our prediction functions to make the position adjustment for the above players. We had to restart the kernel to pick up the new prediction file form S3 so we simply display the predictions before position changes.

In [38]:
old_predictions_players_who_changed_position

,name,name_new,model,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,position
12,anthony_martial,anthony_martial,DeepFantasyFootball_v01,4.499953,4.105354,4.164009,4.055518,3.967909,20.792742,0.0,1.0,0.0,0.0,FWD
15,pierre-emerick_aubameyang,pierre-emerick_aubameyang,DeepFantasyFootball_v01,3.897748,4.059285,3.870273,3.847038,3.696694,19.371038,0.0,0.0,0.0,1.0,MID
16,michail_antonio,michail_antonio,DeepFantasyFootball_v01,3.471481,3.811976,4.013702,4.029356,4.043107,19.369621,0.0,1.0,0.0,0.0,FWD
22,marcus_rashford,marcus_rashford,DeepFantasyFootball_v01,3.857534,3.467133,3.514597,3.449275,3.403746,17.692284,0.0,0.0,0.0,1.0,MID
59,mason_greenwood,mason_greenwood,DeepFantasyFootball_v01,3.280328,2.741984,2.706571,2.626166,2.598459,13.953507,0.0,0.0,0.0,1.0,MID
63,richarlison_de_andrade,richarlison_de andrade,DeepFantasyFootball_v01,2.543257,2.957338,2.733490,2.720685,2.595165,13.549934,0.0,1.0,0.0,0.0,FWD
77,matt_ritchie,matt_ritchie,DeepFantasyFootball_v01,2.599561,2.631610,2.485099,2.421770,2.352202,12.490243,0.0,0.0,0.0,1.0,MID
122,john_lundstram,john_lundstram,DeepFantasyFootball_v01,2.469977,2.151511,2.080721,2.012635,2.014437,10.729283,0.0,0.0,0.0,1.0,MID
192,ben_johnson,ben_johnson,DeepFantasyFootball_v01,2.112235,1.781218,1.669672,1.573829,1.568440,8.705395,1.0,0.0,0.0,0.0,DEF
194,william_smallbone,william_smallbone,DeepFantasyFootball_v01,1.978167,1.827264,1.682684,1.621893,1.531235,8.641242,0.0,0.0,0.0,1.0,MID


In [8]:
players_who_changed_position = {
    'anthony_martial': 'FWD',
    'pierre-emerick_aubameyang': 'MID',
    'michail_antonio': 'FWD',
    'marcus_rashford': 'MID',
    'mason_greenwood': 'MID',
    'richarlison_de andrade': 'FWD',
    'matt_ritchie': 'MID',
    'john_lundstram': 'MID',
    'ben_johnson': 'DEF',
    'william_smallbone': 'MID',
    'fernando_luiz rosa': 'DEF',
    'takumi_minamino': 'MID',
    'eric_dier': 'DEF',
    'keinan_davis': 'FWD',
    'anthony_gordon': 'MID',
    'kelland_watts': 'DEF',
    'gabriel_teodoro martinelli silva': 'MID',
    'kouassi_ryan sessegnon': 'DEF',
    'callum_robinson': 'FWD'
}

In [7]:
predictions_after_position_change = pq.read_table(PREDICTIONS_GW1_S3_PATH, filesystem=s3_filesystem).to_pandas()
predictions_after_position_change.head()

,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model
0,5.657178,5.443412,5.453287,5.351354,5.367841,27.273075,mohamed_salah,0.0,0.0,0.0,1.0,Liverpool,12.0,mohamed_salah,5.079660,5.094854,5.213835,5.167535,5.035730,25.591616,mohamed_salah,0.0,0.0,0.0,1.0,mohamed_salah,mohamed_salah,both,5.079660,5.094854,5.213835,5.167535,5.035730,25.591616,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
1,6.319860,6.077927,6.090129,5.983685,6.004195,30.475794,raheem_sterling,0.0,0.0,0.0,1.0,Manchester City,11.5,raheem_sterling,4.751492,5.080462,4.870038,4.858727,4.749552,24.310270,raheem_sterling,0.0,0.0,0.0,1.0,raheem_sterling,raheem_sterling,both,4.751492,5.080462,4.870038,4.858727,4.749552,24.310270,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
2,4.679602,4.493706,4.491284,4.391965,4.398650,22.455206,harry_kane,0.0,1.0,0.0,0.0,Tottenham Hotspur,10.5,harry_kane,4.645383,4.666142,4.685612,4.617728,4.522790,23.137653,harry_kane,0.0,1.0,0.0,0.0,harry_kane,harry_kane,both,4.645383,4.666142,4.685612,4.617728,4.522790,23.137653,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
3,4.239182,4.072062,4.067756,3.971863,3.975711,20.326572,andrew_robertson,1.0,0.0,0.0,0.0,Liverpool,7.0,andrew_robertson,5.091536,4.405899,4.418815,4.247555,4.096176,22.259981,andrew_robertson,1.0,0.0,0.0,0.0,andrew_robertson,andrew_robertson,both,5.091536,4.405899,4.418815,4.247555,4.096176,22.259981,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01
4,4.516146,4.338603,4.335416,4.238196,4.243636,21.671997,pierre-emerick_aubameyang,0.0,1.0,0.0,0.0,Arsenal,12.0,pierre-emerick_aubameyang,4.203787,4.467486,4.323889,4.305782,4.171481,21.472424,pierre-emerick_aubameyang,0.0,0.0,0.0,1.0,pierre-emerick_aubameyang,pierre-emerick_aubameyang,both,4.203787,4.467486,4.323889,4.305782,4.171481,21.472424,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01


In [10]:
predictions_after_position_change[predictions_after_position_change['name'].isin(players_who_changed_position.keys())]

,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model
4,4.516146,4.338603,4.335416,4.238196,4.243636,21.671997,pierre-emerick_aubameyang,0.0,1.0,0.0,0.0,Arsenal,12.0,pierre-emerick_aubameyang,4.203787,4.467486,4.323889,4.305782,4.171481,21.472424,pierre-emerick_aubameyang,0.0,0.0,0.0,1.0,pierre-emerick_aubameyang,pierre-emerick_aubameyang,both,4.203787,4.467486,4.323889,4.305782,4.171481,21.472424,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
13,4.102262,3.938238,3.932136,3.836074,3.838531,19.647242,marcus_rashford,0.0,1.0,0.0,0.0,Manchester United,9.5,marcus_rashford,4.253430,3.990873,4.064881,4.003408,3.954192,20.266785,marcus_rashford,0.0,0.0,0.0,1.0,marcus_rashford,marcus_rashford,both,4.253430,3.990873,4.064881,4.003408,3.954192,20.266785,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
20,4.678135,4.494624,4.493752,4.395195,4.402573,22.464281,anthony_martial,0.0,0.0,0.0,1.0,Manchester United,9.0,anthony_martial,4.067800,3.555353,3.580412,3.467710,3.378774,18.050051,anthony_martial,0.0,1.0,0.0,0.0,anthony_martial,anthony_martial,both,4.067800,3.555353,3.580412,3.467710,3.378774,18.050051,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
34,3.758688,3.607759,3.600152,3.506060,3.506362,17.979021,michail_antonio,0.0,0.0,0.0,1.0,West Ham United,6.5,michail_antonio,2.967970,3.170918,3.334451,3.341283,3.357939,16.172560,michail_antonio,0.0,1.0,0.0,0.0,michail_antonio,michail_antonio,both,2.967970,3.170918,3.334451,3.341283,3.357939,16.172560,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
37,3.427543,3.287133,3.274909,3.182421,3.179076,16.351082,mason_greenwood,0.0,1.0,0.0,0.0,Manchester United,7.5,mason_greenwood,3.568909,3.152601,3.140765,3.072471,3.032846,15.967592,mason_greenwood,0.0,0.0,0.0,1.0,mason_greenwood,mason_greenwood,both,3.568909,3.152601,3.140765,3.072471,3.032846,15.967592,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
84,2.846878,2.726928,2.709986,2.620365,2.612378,13.516536,matt_ritchie,1.0,0.0,0.0,0.0,Newcastle United,5.0,matt_ritchie,2.527095,2.590272,2.430125,2.359848,2.279669,12.187009,matt_ritchie,0.0,0.0,0.0,1.0,matt_ritchie,matt_ritchie,both,2.527095,2.590272,2.430125,2.359848,2.279669,12.187009,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
133,2.955618,2.834475,2.819997,2.731657,2.725289,14.067035,john_lundstram,1.0,0.0,0.0,0.0,Sheffield United,5.5,john_lundstram,2.318158,2.143414,2.047753,1.993147,1.962366,10.464837,john_lundstram,0.0,0.0,0.0,1.0,john_lundstram,john_lundstram,both,2.318158,2.143414,2.047753,1.993147,1.962366,10.464837,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
188,1.611261,1.551921,1.558099,1.498080,1.501280,7.720642,william_smallbone,0.0,0.0,0.0,1.0,Southampton,4.5,william_smallbone,1.981032,1.884733,1.760198,1.707522,1.632002,8.965487,william_smallbone,0.0,0.0,0.0,1.0,william_smallbone,william_smallbone,both,1.981032,1.884733,1.760198,1.707522,1.632002,8.965487,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
190,1.818829,1.746277,1.742000,1.672429,1.669525,8.649060,ben_johnson,1.0,0.0,0.0,0.0,West Ham United,4.0,ben_johnson,2.224593,1.781411,1.712197,1.589753,1.620592,8.928547,ben_johnson,1.0,0.0,0.0,0.0,ben_johnson,ben_johnson,both,2.224593,1.781411,1.712197,1.589753,1.620592,8.928547,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01
195,1.188663,1.166677,1.213533,1.186252,1.214184,5.969310,takumi_minamino,0.0,0.0,0.0,1.0,Liverpool,6.5,takumi_minamino,1.961383,1.719880,1.707573,1.678667,1.647805,8.715308,takumi_minamino,0.0,0.0,0.0,1.0,takumi_minamino,takumi_minamino,both,1.961383,1.719880,1.707573,1.678667,1.647805,8.715308,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01


Observations:
- Players who went from FWD to MID, generally saw a points increase (which is to be expected)
- Conversely players who went from MID to FWD saw a decrease (Anthony Martial)
- MID to DEF led to a slight increase (Eric Dier) or decrease in the opposite direction (John Lundstram)
- Aubamayang jumped to 5th overall in predictions! Can see why he's in high demand. He is however the exact same price as Salah who tops the standings.